In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install gitingest
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 13.2 MB/s eta 0:00:00


In [3]:
import re
import asyncio
from gitingest import ingest_async
from pinecone import Pinecone

In [8]:
summary, tree, content = await ingest_async("https://github.com/coder-zs-cse/zencode-frontend")

In [9]:
sections = re.split(r'={48}', content)

In [10]:
fetchedComponents = []
# Iterate through the sections and extract file names,file contents and paths
j = 0
for i in range(1, len(sections), 2):
   path = sections[i].strip().split(': ')[1]
   file_content = sections[i + 1].strip()
   fetchedComponents.append({'file': path.split("/")[-1],'fileContent':file_content,'path':path})
   j = j + 1


In [4]:
from openai import OpenAI
import json
import uuid

In [13]:

# # for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
# response = client.chat.completions.create(
#     model="deepseek-chat",
#     messages=[
#         {"role": "system", "content": systemPrompt},
#         {"role": "user", "content": basePrompt+"\n the location of the file in the external project is: "},
#   ],
#     max_tokens=1024,
#     temperature=0.7,
#     stream=False
# )

# print(response.choices[0].message.content)

In [ ]:
client = OpenAI(api_key="sk-fff", base_url="https://api.deepseek.com")

In [15]:
systemPrompt = """
You are a expert full stack developer with expertise in node js, react js and next js. output all responses in JSON blobs only
"""

In [16]:
basePrompt="""
given below is a javascript file of a react js project, analyse and extract all components that can be exported. generate a json blob doe every exportable components consisting of the following fields 'name':name of the component,  'description': a detailed description of what the component does, 'returnType': what the component returns, 'inputParams': list of all input variable type, name and description, incase of c class or interface mention in the variable type an description, 'import': how to import the component from another project, 'example': brief code example importing and using the component in another project always return the response in list of json objects   
"""

In [17]:
jsonChunks = []

In [19]:
for component in fetchedComponents:
    if component['path'].startswith('src'):
        # print("File:", component['file'])
        # print("File Content:", component['fileContent'])
        print("Path:", component['path'])
        
        # for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.

        userPrompt = basePrompt+"here is the file content"+component['fileContent']+"\n the location of the file in the external project is: "+component['path']
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": systemPrompt},
                {"role": "user", "content": userPrompt},
          ],
            max_tokens=6000,
            temperature=0,
            stream=False
        )

        json_match = re.search(r'\[.*\]', response.choices[0].message.content, re.DOTALL)
        if json_match:
            json_part = json_match.group()
        else:
            print("No JSON found in the string.")
            json_part = None
        
        if json_part:
            json_list = json.loads(json_part)
            print("Appended JSON objects:")
            for obj in json_list:
                print(obj)
                jsonChunks.append(obj)
        
                

Path: src/api/axios.ts
Appended JSON objects:
{'name': 'axiosSecure', 'description': 'An Axios instance configured with secure settings, including request and response interceptors for handling authentication tokens.', 'returnType': 'AxiosInstance', 'inputParams': [], 'import': "import { axiosSecure } from 'src/api/axios';", 'example': "import { axiosSecure } from 'src/api/axios';\n\naxiosSecure.get('/some-endpoint').then(response => {\n  console.log(response.data);\n}).catch(error => {\n  console.error(error);\n});"}
{'name': 'axiosOpen', 'description': 'An Axios instance configured without secure settings, suitable for public API requests that do not require authentication.', 'returnType': 'AxiosInstance', 'inputParams': [], 'import': "import { axiosOpen } from 'src/api/axios';", 'example': "import { axiosOpen } from 'src/api/axios';\n\naxiosOpen.get('/public-endpoint').then(response => {\n  console.log(response.data);\n}).catch(error => {\n  console.error(error);\n});"}
Path: src/ap

In [20]:
jsonChunks[1]

{'name': 'axiosOpen',
 'description': 'An Axios instance configured without secure settings, suitable for public API requests that do not require authentication.',
 'returnType': 'AxiosInstance',
 'inputParams': [],
 'import': "import { axiosOpen } from 'src/api/axios';",
 'example': "import { axiosOpen } from 'src/api/axios';\n\naxiosOpen.get('/public-endpoint').then(response => {\n  console.log(response.data);\n}).catch(error => {\n  console.error(error);\n});"}

In [ ]:
pc = Pinecone("fff")
index = pc.Index("llama-text-embed-v2-index")

In [35]:
embeddings = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[d['description'] for d in jsonChunks],
    parameters={
        "input_type": "passage"
    }
)

vectors = []
for d, e in zip(jsonChunks, embeddings):
    vectors.append({
        "id": str(uuid.uuid1()),
        "values": e['values'],
        "metadata": {"component":json.dumps(d)}
    })

In [36]:
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 69}

In [38]:
# !pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [39]:
# from smolagents import Tool, HfApiModel, ToolCallingAgent

In [40]:
# class RetrieverTool(Tool):
#     name = "retriever"
#     description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
#     inputs = {
#         "query": {
#             "type": "string",
#             "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
#         }
#     }
#     output_type = "string"

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

#     def forward(self, query: str) -> str:
#         assert isinstance(query, str), "Your search query must be a string"
#         queryEmbedding = pc.inference.embed(
#             model="llama-text-embed-v2",
#             inputs=[query],
#             parameters={
#                 "input_type": "passage"
#             }
#         )
        
#         docs = index.query(            
#             namespace="ns1",
#             vector=queryEmbedding[0]['values'],
#             top_k=3,
#             include_values=True,
#             include_metadata=True,
# )

#         return "\nRetrieved documents:\n" + "".join(
#             [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
#         )

In [ ]:
!pip install -U "huggingface_hub[cli]" -q
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('fff')"

In [76]:


# def generate_response(prompt):
#     response = client.chat.completions.create(
#             model="deepseek-chat",
#             messages=[
#                 {"role": "system", "content": "You are a helpful assistant"},
#                 {"role": "user", "content": prompt},
#           ],
#             max_tokens=500,
#             temperature=0,
#             stream=False
#         )
#     return response["choices"][0]["text"].strip()

In [77]:
# model = HfApiModel("meta-llama/Llama-3.1-70B-Instruct")

# retriever_tool = RetrieverTool()
# agent = ToolCallingAgent(tools=[retriever_tool], model=generate_response)
# agent.run("fetch a component to create a navbar")

In [10]:
def generate_multi_query(query, model="deepseek-chat"):

    prompt = """
    You are a knowledgeable full stack developer. 
    Your need to find the most appropriate component in an exteranl library. 
    For the given question, propose up to five related questions to assist them in finding the right component you need. 
    Provide concise, single-topic questions (withouth compounding sentences) that cover various aspects of the topic. 
    Ensure each question is complete and directly related to the original inquiry. 
    List each question on a separate line without numbering.
                """

    messages = [
        {
            "role": "system",
            "content": prompt,
        },
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content


original_query = (
    "fetch a component to create a navbar"
)
aug_queries = generate_multi_query(original_query)

# 1. First step show the augmented queries
for query in aug_queries:
    print("\n", query)





 What type of navigation structure do you need (horizontal, vertical, or responsive)?  

 Do you prefer a minimalist design or one with more visual elements?  

 Should the navbar include dropdown menus or submenus?  

 Are you looking for a mobile-friendly navbar with a hamburger menu?  

 Do you need built-in support for active link highlighting?


In [24]:
joint_query = [
    original_query
] + aug_queries  # original query is in a list because chroma can actually handle multiple queries, so we add it in a list

# print("======> \n\n", joint_query)
queryEmbedding = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=joint_query,
    parameters={
        "input_type": "passage"
    }
)

response = index.query(            
    namespace="ns1",
    vector=queryEmbedding[0]['values'],
    top_k=3,
    include_values=True,
    include_metadata=True,
    )
    


In [31]:
response['matches'][0]['metadata']

{'component': '{"name": "Title", "description": "A React functional component that renders a styled heading with the provided text.", "returnType": "JSX.Element", "inputParams": [{"type": "TitleProps", "name": "text", "description": "The text to be displayed inside the heading."}], "import": "import { Title } from \'src/components/ui/title/title\';", "example": "import { Title } from \'src/components/ui/title/title\';\\n\\nconst App = () => (\\n  <div>\\n    <Title text=\\"Welcome to My App\\" />\\n  </div>\\n);"}'}

In [37]:
def word_wrap(text, width=87):
    """
    Wraps the given text to the specified width.

    Args:
    text (str): The text to wrap.
    width (int): The width to wrap the text to.

    Returns:
    str: The wrapped text.
    """
    return "\n".join([text[i : i + width] for i in range(0, size(text), width)])

In [41]:
# print(response)
retrieved_documents = response["matches"]

# Deduplicate the retrieved documents
unique_documents = set()
for document in retrieved_documents:
    # for document in documents:
        unique_documents.add(document['id'])


In [42]:
unique_documents

{'84d3a1a8-0881-11f0-8b6a-0242ac130202',
 '84d3a892-0881-11f0-8b6a-0242ac130202',
 '84d3ac48-0881-11f0-8b6a-0242ac130202'}

In [44]:
def augment_query_generated(query, model="deepseek-chat"):
    prompt = """You are a helpful expert fullstack developer. 
   Provide an example answer to the given question, that might be found in a code description like a library."""
    messages = [
        {
            "role": "system",
            "content": prompt,
        },
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content


original_query = "fetch a component to create a navbar"
hypothetical_answer = augment_query_generated(original_query)

joint_query = f"{original_query} {hypothetical_answer}"